In [3]:
# ── 필수 라이브러리 ─────────────────────────────────────────────
import json, folium, pandas as pd, matplotlib.pyplot as plt
from folium.plugins import FloatImage


# ── ① 엑셀·GeoJSON 읽기 (경로는 Colab 기본 /content 기준) ──
df  = pd.read_excel('/Users/sullem/yj/HIuniv_Project/results/seokjun/SIG_geojson_위도경도_백분율.xlsx')
with open('/Users/sullem/yj/HIuniv_Project/results/seokjun/SIG.geojson', encoding='utf-8') as f:
    geo = json.load(f)

In [4]:
# ── ② 데이터 전처리 ───────────────────────────────────────────
df['SIG_CD']        = df['SIG_CD'].astype(str)           # 문자열로 변환
bins   = [0, 20, 40, 60, 80, 100]                        # 5 구간
labels = [1, 2, 3, 4, 5]
df['강수등급']       = pd.cut(df['백분율'], bins=bins, labels=labels,
                        include_lowest=True).astype(int)
df['강수위험도(%)']   = df['백분율'].round(2)               # 소수점 2자리


In [5]:
# ── ③ GeoJSON 속성에 df 값 머지(팝업/툴팁용) ────────────────
code_map = df.set_index('SIG_CD')[['강수위험도(%)','강수등급']].to_dict('index')
for feat in geo['features']:
    code = str(feat['properties']['SIG_CD'])
    if code in code_map:
        feat['properties'].update(code_map[code])


In [6]:
# ── ④ 지도 & Choropleth ─────────────────────────────────────
m = folium.Map(location=[36.5, 127.8], zoom_start=7)

folium.Choropleth(
    geo_data      = geo,
    data          = df,
    columns       = ['SIG_CD','백분율'],
    key_on        = 'feature.properties.SIG_CD',
    fill_color    = 'YlGnBu',
    fill_opacity  = 0.8,
    line_opacity  = 0.4,
    nan_fill_color='white',
    legend_name   = '통합 백분위 (5-구간)',
    bins          = bins,
    reset         = True
).add_to(m)



In [7]:
# ── ⑤ 클릭 툴팁 & 팝업 ──────────────────────────────────────
folium.GeoJson(
    geo,
    style_function     = lambda _: {'fillOpacity':0, 'weight':0},
    highlight_function = lambda _: {'weight':2, 'color':'#666'},
    tooltip = folium.GeoJsonTooltip(
        fields  = ['SIG_KOR_NM','강수위험도(%)','강수등급'],
        aliases = ['행정동','강수위험도(%)','강수등급'],
        localize=True
    ),
    popup   = folium.GeoJsonPopup(
        fields  = ['SIG_KOR_NM','강수위험도(%)','강수등급'],
        aliases = ['행정동','강수위험도(%)','강수등급'],
        localize=True
    )
).add_to(m)


In [8]:
# ⑥-A 상위 10개 지역 막대그래프 HTML 생성 & 지도에 고정 ──────────────
from branca.element import MacroElement
from jinja2 import Template

# 1) 상위 10개 추출 (값 ↑순으로 정렬)
top10 = (df.nlargest(10, '백분율')
           .sort_values('백분율'))

# 2) HTML 막대그래프 만들기  ────────────────
rows = ""
for _, r in top10.iterrows():
    rows += f"""
    <tr>
      <td style='padding-right:6px; white-space:nowrap; font-size:12px;'>
          {r['SIG_KOR_NM']}
      </td>
      <td style='width:100%;'>
          <div style='height:8px; background:#d65f9a; width:{r['백분율']:.2f}%;'></div>
      </td>
      <td style='padding-left:4px; font-size:12px;'>{r['백분율']:.2f}</td>
    </tr>
    """

chart_html = f"""
<h4 style='margin:0 0 6px 0; font-size:14px;'>상위 10개 강수위험도(%)</h4>
<table style='border-collapse:collapse; width:300px;'>{rows}</table>
"""

# 3) 지도에 “고정 오버레이”로 삽입  ──────────
class FixedHtmlOverlay(MacroElement):
    """지도 화면 위 고정(over-div) HTML 표시용"""
    _template = Template(u"""
    {% macro html(this, kwargs) %}
    <div style="
         position: fixed;
         bottom: 10px;              /* ↕ 위치 조정 */
         right: 10px;               /* ↔ 위치 조정 */
         z-index: 9999;
         background: rgba(255,255,255,0.95);
         padding: 8px 10px;
         border: 1px solid #777;
         border-radius: 4px;
         box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
         ">
      {{ this.html|safe }}
    </div>
    {% endmacro %}
    """)  # noqa: E501

    def __init__(self, html):
        super().__init__()
        self.html = html

# 4) 지도 객체(m)에 추가
m.get_root().add_child(FixedHtmlOverlay(chart_html))

m.save('map_with_chart.html') 
